In [1]:
import pandas as pd
import json
import xlsxwriter
import csv
import numpy as np

In [2]:
bdd_table = pd.ExcelFile("Output_of_BDD_States_final.xlsx")
commands_list=[]
for idx, name in enumerate(bdd_table.sheet_names):
    #print(idx,name)
    table = pd.read_excel("Output_of_BDD_States_final.xlsx", sheet_name=name, startrow=0, startcol=0,index=None)
    dict={}
    if(name=='sEth'):
              
        temp_dict={}
        temp_dict.update({'exact':[],'ternary':[],'default':[]})
        
        for index, row in table.iterrows():
            if(row[name]=='*'):
                temp_dict.get('ternary').append(row)
            else:
                temp_dict.get('exact').append(row)
          

        for y in temp_dict.items():

            if(y[0]=='exact'):
                str1 = 'table_add '+name+'_'+y[0]+' store_state_'+name+' '
                for z in temp_dict.get(y[0]):
                    str2 =str1+z[0]+' => '+str(z[1])
                    commands_list.append(str2)
                #print(type(z[0]))

            elif(y[0]=='ternary'):
                str4 = 'table_add '+name+'_'+y[0]+' store_state_'+name+' '
                for z in temp_dict.get(y[0]):
                    str5 =str4+'0x0&&&0x0 => '+str(z[1])+' 1'
                    commands_list.append(str5)
    elif(name=='action'):
        
        for index, row in table.iterrows():
            if(row[1]=='forward'):
                str1 = 'table_add final '+row[1]+' '+str(row[0])+' => 00:00:00:00:00:00 1'
            else:
                str1 = 'table_add final '+row[1]+' '+str(row[0])
        
            commands_list.append(str1)
            
            
    else:
       
        #all columns/tab except sEth & action
        for i in np.unique(table['Previous State'].tolist()):
            dict.update({str(i):[]})
        
        for k in dict.keys():
            for i in range(0,table.shape[0]):
                dict.update({k:table.loc[table['Previous State']==int(k),name].tolist()})
            #print(dict)
        temp_dict={}
        temp_dict.update({'exact':[],'ternary':[],'default':[]})
        temp_list=[]

        for x in dict.items():

            if '*' in dict.get(x[0]):
                if(len(dict.get(x[0]))>1):
                    temp_dict.get('ternary').append(x)

                else:
                    temp_dict.get('default').append(x)

            else:
                temp_dict.get('exact').append(x)

        #print(temp_dict)


        for y in temp_dict.items():

            if(y[0]=='exact'):
                if(name=='Source' or name == 'Destination'):
                    str1 = 'table_add '+name+'_'+y[0]+' store_state_'+name+' '
                    for z in temp_dict.get(y[0]):
                        str2 =str1+z[0]
                    #print(type(z[0]))
                        for w in z[1]:
                            str3 = str2+' 0x'+w+' '+'=>'
                            #print(str3)
                            value =table.loc[(table['Previous State']==int(z[0]))&(table[name]==str(w)),'Next State'].to_string(index=False)
                            #print(str3+''+value)
                            commands_list.append(str3+''+value)
                else:
                    str1 = 'table_add '+name+'_'+y[0]+' store_state_'+name+' '
                    for z in temp_dict.get(y[0]):
                        str2 =str1+z[0]
                    #print(type(z[0]))
                        for w in z[1]:
                            str3 = str2+' '+w+' '+'=>'
                            #print(str3)
                            value =table.loc[(table['Previous State']==int(z[0]))&(table[name]==str(w)),'Next State'].to_string(index=False)
                            #print(str3+''+value)
                            commands_list.append(str3+''+value)
            elif(y[0]=='ternary'):
                if(name=='typEth' or name=='proto' or name =='sPort' or name == 'dPort'):
                    str4 = 'table_add '+name+'_'+y[0]+' store_state_'+name+' '
                    for z in temp_dict.get(y[0]):
                        str5 =str4+z[0]
                        dd=sorted(z[1],reverse=True)
                        counter=0
                        for w in dd:
                            counter=counter+1
                            #raw_df2['Priority'] = range(1, 1+len(raw_df2))
                            #print(str3)
                            if(w!='*'):
                                str6 = str5+' 0x'+w+'&&&0x0ff '+'=>'
                            else:
                                str6 = str5+' 0x0&&&0x0 '+'=>'
                            value =table.loc[(table['Previous State']==int(z[0]))&(table[name]==str(w)),'Next State'].to_string(index=False)

                            #print(str6+''+value+' '+str(counter))
                            commands_list.append(str6+''+value+' '+str(counter))
                else:
                    str4 = 'table_add '+name+'_'+y[0]+' store_state_'+name+' '
                    for z in temp_dict.get(y[0]):
                        str5 =str4+z[0]
                        dd=sorted(z[1],reverse=True)
                        counter=0
                        for w in dd:
                            counter=counter+1
                            #raw_df2['Priority'] = range(1, 1+len(raw_df2))
                            #print(str3)
                            if(w!='*'):
                                str6 = str5+' 0x'+w+'&&&0x0fff '+'=>'
                            else:
                                str6 = str5+' 0x0&&&0x0 '+'=>'
                            value =table.loc[(table['Previous State']==int(z[0]))&(table[name]==str(w)),'Next State'].to_string(index=False)

                            #print(str6+''+value+' '+str(counter))
                            commands_list.append(str6+''+value+' '+str(counter))
            elif(y[0]=='default'):
                str7 = 'table_add '+name+'_'+y[0]+' store_state_'+name+' '
                for z in temp_dict.get(y[0]):
                    str8 =str7+z[0]

                    for w in z[1]:
                        str9 = str8+' '+'=>'

                        value =table.loc[table['Previous State']==int(z[0]),'Next State'].to_string(index=False)
                        #print(str9+''+value)
                        commands_list.append(str9+''+value)
        
#print(commands_list)

In [3]:
with open('BDD_s1commands_test.txt', 'w') as f:
    for item in commands_list:
        f.write("%s\n" % item)